## Generate camera data files for instrument configuration

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import bossdata
import fitsio
import os
import astropy.units as u

from astropy.io import fits
from specsim import config, instrument

In [3]:
finder = bossdata.path.Finder()
mirror = bossdata.remote.Manager()

In [ ]:
fwhm = np.sqrt(8*np.log(2))

def get_hdulist(sequence_number, band, fiber, plan_path):
    local_plan_path = mirror.get(plan_path)
    Plan = bossdata.plate.Plan(local_plan_path)
    exp_name = Plan.get_exposure_name(sequence_number=sequence_number, band=band, fiber=fiber, ftype='spCFrame')
    spec_index = Plan.get_spectrograph_index(fiber=fiber)
    local_fits = mirror.get(os.path.join(plan_path.strip(os.path.basename(plan_path)), exp_name))
    hdulist = fits.open(local_fits)
    Frame = bossdata.plate.FrameFile(path=local_fits, index=spec_index, calibrated=True)
    return(hdulist, Frame.camera)


def plot_cameras(plate, mjd, sequence_number=1, rms=False, plot=True):
    plan_path = finder.get_plate_plan_path(plate=plate, mjd=mjd, combined=True)
    b1_hdulist, b1_title = get_hdulist(sequence_number=sequence_number, band='blue', fiber=30, plan_path=plan_path)
    b2_hdulist, b2_title = get_hdulist(sequence_number=sequence_number, band='blue', fiber=600, plan_path=plan_path)
    r1_hdulist, r1_title = get_hdulist(sequence_number=sequence_number, band='red', fiber=30, plan_path=plan_path)
    r2_hdulist, r2_title = get_hdulist(sequence_number=sequence_number, band='red', fiber=600, plan_path=plan_path)
    if plot:
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(15, 11))
        b1_w = 10**b1_hdulist[3].data
        b1_r = b1_hdulist[4].data*0.0001*np.log(10)*b1_w
        b2_w = 10**b2_hdulist[3].data
        b2_r = b2_hdulist[4].data*0.0001*np.log(10)*b2_w
        r1_w = 10**r1_hdulist[3].data
        r1_r = r1_hdulist[4].data*0.0001*np.log(10)*r1_w
        r2_w = 10**r2_hdulist[3].data
        r2_r = r2_hdulist[4].data*0.0001*np.log(10)*r2_w
        
        if rms:
            ax1.scatter(b1_w, b1_r, s=3)
            ax2.scatter(b2_w, b2_r, s=3)
            ax3.scatter(r1_w, r1_r, s=3)
            ax4.scatter(r2_w, r2_r, s=3)
            ax1.set_ylabel('RMS Resolution (Angstrom)')
            ax2.set_ylabel('RMS Resolution (Angstrom)')
            ax3.set_ylabel('RMS Resolution (Angstrom)')
            ax4.set_ylabel('RMS Resolution (Angstrom)')
            
        else:
            ax1.scatter(b1_w, b1_r*fwhm, s=3)
            ax2.scatter(b2_w, b2_r*fwhm, s=3)
            ax3.scatter(r1_w, r1_r*fwhm, s=3)
            ax4.scatter(r2_w, r2_r*fwhm, s=3)
            ax1.set_ylabel('FWHM Resolution (Angstrom)')
            ax2.set_ylabel('FWHM Resolution (Angstrom)')
            ax3.set_ylabel('FWHM Resolution (Angstrom)')
            ax4.set_ylabel('FWHM Resolution (Angstrom)')




        
        ax1.set_title('{:s}'.format(b1_title))
        ax1.set_xlabel('Wavelength (Angstrom)')
        ax2.set_title('{:s}'.format(b2_title))
        ax2.set_xlabel('Wavelength (Angstrom)')
        ax3.set_title('{:s}'.format(r1_title))
        ax3.set_xlabel('Wavelength (Angstrom)')
        ax4.set_title('{:s}'.format(r2_title))
        ax4.set_xlabel('Wavelength (Angstrom)')
        
        
        plt.tight_layout()
        #plt.savefig('plate7301_mjd56746.png')
        plt.show()
        
    return b1_hdulist, b2_hdulist, r1_hdulist, r2_hdulist